# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [4]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [5]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [6]:
describe(boston)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Crim,3.61352,0.00632,0.25651,88.9762,,,Float64
2,Zn,11.3636,0.0,0.0,100.0,,,Float64
3,Indus,11.1368,0.46,9.69,27.74,,,Float64
4,Chas,0.06917,0,0.0,1,,,Int64
5,NOx,0.554695,0.385,0.538,0.871,,,Float64
6,Rm,6.28463,3.561,6.2085,8.78,,,Float64
7,Age,68.5749,2.9,77.5,100.0,,,Float64
8,Dis,3.79504,1.1296,3.20745,12.1265,,,Float64
9,Rad,9.54941,1,5.0,24,,,Int64


###Variables

#There are 14 attributes in each case of the dataset. They are:
#CRIM - per capita crime rate by town
#ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
#INDUS - proportion of non-retail business acres per town.
#CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
#NOX - nitric oxides concentration (parts per 10 million)
#RM - average number of rooms per dwelling
#AGE - proportion of owner-occupied units built prior to 1940
#DIS - weighted distances to five Boston employment centres
#RAD - index of accessibility to radial highways
#TAX - full-value property-tax rate per 10,000
#PTRATIO - pupil-teacher ratio by town
#B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#LSTAT - % lower status of the population
#MEDV - Median value of owner-occupied homes in 1000's

In [10]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [12]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,10.6718,0.0,18.1,0,0.74,6.459,94.8,1.9879,24,666
2,0.02543,55.0,3.78,0,0.484,6.696,56.4,5.7321,5,370
3,0.07896,0.0,12.83,0,0.437,6.273,6.0,4.2515,5,398
4,0.04011,80.0,1.52,0,0.404,7.287,34.1,7.309,2,329
5,0.11504,0.0,2.89,0,0.445,6.163,69.6,3.4952,2,276
6,0.09744,0.0,5.96,0,0.499,5.841,61.4,3.3779,5,279
7,0.21719,0.0,10.59,1,0.489,5.807,53.8,3.6526,4,277
8,0.03705,20.0,3.33,0,0.4429,6.968,37.2,5.2447,5,216
9,0.10659,80.0,1.91,0,0.413,5.936,19.5,10.5857,4,334


In [13]:
ols = GLM.lm(@formula(MedV ~ Crim + Zn + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Crim + Zn + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                 Estimate  Std. Error   t value  Pr(>|t|)     Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────────
(Intercept)   32.1145      5.52159      5.81618    <1e-7    21.259       42.9701    
Crim          -0.118175    0.0330255   -3.57829    0.0004   -0.183104    -0.0532462 
Zn             0.0439132   0.0149243    2.9424     0.0034    0.0145718    0.0732545 
Chas           3.90752     0.922451     4.23601    <1e-4     2.09396      5.72107   
NOx          -16.9983      3.78047     -4.49633    <1e-5   -24.4307      -9.56578   
Rm             4.23576     0.446389     9.48894    <

In [15]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 12.398018779577939
 28.037517080270046
 25.646513619931458
 36.568876654294016
 24.769097963098087
 22.108977829185136
 23.742762758001696
 34.198184694541524
 16.805559578070007
 31.373124869888784
 28.386417238258236
 26.9317029976127  
 15.287992810239864
  ⋮                
  9.964748269697154
 17.448459155313433
 20.41449584534373 
 26.30585649050937 
 15.83577817482222 
 26.261491251775027
 33.66732954213812 
  9.123940291601862
 25.236663200920752
 17.380080224964836
 23.47484331000123 
 42.13790454127581 

In [16]:
GLM.r²(ols)

0.7544467243489484

In [17]:
GLM.adjr²(ols)

0.7475737319007002